In [3]:
from random import sample, seed

import pandas as pd

In [4]:
pd.options.display.max_colwidth = 1000

In [6]:
df = pd.read_csv('D:\\University\\вкр магистр\\Исходные\\Epicrises\\example\\epicrises_text.csv')

df_clean = df
df_clean = df_clean.fillna('')  

In [7]:
df_main = pd.DataFrame()

In [8]:
df_clean['id'] = list(range(1, len(df_clean) + 1))
df_main['id'] = df_clean['id']

In [9]:
df_main['diagnosis'] = df_clean['diagnosis']
df_main['complaint'] = df_clean['complaint']
df_main['anamnesis'] = df_clean['anamnesis']
df_main['status_end'] = df_clean['status_end']
df_main['disease_outcome'] = df_clean['disease_outcome']

In [10]:
df_main.sample(5)

,id,diagnosis,complaint,anamnesis,status_end,disease_outcome
1024,1025,"Основной: Коронавирусная инфекция CОVID-19 (вирус не идентифицирован), средне-тяжелая форма. Осложнение: Внебольничная двусторонняя полисегментарная пневмония. ДН 0-IСопутствующий: ИБС. Стенокардия напряжения ФК 2. Коронарное шунтирование: МКШ-ПНА, ВТК 29.06.20г.ХСН 1, ФК 3.Гипертоническая болезнь стадия 3, степень АГ корригирована, риск 4.","на одышку в покое, слабость, ломоту в теле, повышение температуры до 37,6 С.","Считает себя больным с 01.07.2020 , когда впервые появились вышеперечисленные жалобы. Обратился в Клинику БГМУ, где проведено КТ ОГК, выявлены изменения в легких с обеих сторон по типу матового стекла (24% поражения легких). Госпитализируется для консервативного лечения в Инфекционный госпиталь кл БГМУ.",удовлетворительное,с улучшением
1201,1202,"Основной: Коронавирусная инфекция CОVID-19 (вирус не идентифицирован), среднетяжелая форма. \n \nОсложнения: Внебольничная двусторонняя полисегментарная пневмония. ДН 0 ст.","на одышку в покое, слабость, ломоту в теле, кашель сухой с мокротой, повышение температуры до С.","Считает себя больным неделю , когда впервые появились вышеперечисленные жалобы. Не лечилась. 17.06.2020 самостоятельно обратилась в клинику БГМУ, где проведено КТ ОГК выявлены изменения в легких с обеих сторон по типу матового стекла. Госпитализируется для консервативного лечения в инфекционный госпиталь клиники БГМУ.",удовлетворительное,с улучшением
245,246,"Основной: Коронавирусная инфекция CОVID-19 (вирус не идентифицирован), средне-тяжелая форма. Осложнение: Внебольничная двусторонняя полисегментарная пневмония. ДН I","на одышку в покое, слабость, ломоту в теле, кашель сухой, повышение температуры до 38 С.","Считает себя больным с 04.08.2020г., когда впервые появились вышеперечисленные жалобы. Обратился в поликлинику по м/ж, принимал азитромицин, цефтриоксон, парацетамол. Состояние без особой динамики, нарастает одышка. По линии СМП направлен на стационарное лечение в Клинику БГМУ, где проведено КТ ОГК выявлены изменения в легких с обеих сторон по типу матового стекла (64% поражения легких). Госпитализируется для консервативного лечения в Инфекционный госпиталь кл БГМУ.",удовлетворительное,с улучшением
1167,1168,"Основной: Коронавирусная инфекция CОVID-19 (вирус не идентифицирован), средне-тяжелая форма. Осложнение: Внебольничная двусторонняя полисегментарная пневмония. ДН IСопутствующий: ИБС: ПИКС, АКШ от 2015 г.","на одышку в покое, слабость, ломоту в теле, кашель сухой, повышение температуры до 38,0 С","Считает себя больным с 09/10/2020 , когда впервые появились вышеперечисленные жалобы.Обратился в поликлинику по м/ж, принимал ингаверин, парацетамол, азитромицин. Сдан мазок на ПЦР коронавируса от14/10/2020 - результат не известен.Состояние без особой динамики, нарастает одышка. Обратился в Клинику БГМУ, где проведено КТ ОГК выявлены изменения в легких с обеих сторон по типу матового стекла . Госпитализируется для консервативного лечения в Инфекционный госпиталь Клиники БГМУ.",удовлетворительное,с улучшением
55,56,"Основной: Коронавирусная инфекция CОVID-19 (вирус идентифицирован), средне-тяжелая форма. Осложнение: Внебольничная двусторонняя полисегментарная пневмония, средней степени тяжести. ДН 0-IСопутствующий: Гипертоническая болезнь 2 ст, 1 степень риск 3.","на одышку в покое, слабость, ломоту в теле, кашель сухой, повышение температуры до 37,8 С.","Считает себя больным с 14.12.2020 , когда впервые появились вышеперечисленные жалобы.Обратился в поликлинику по м/ж, принимал парацетамол.Состояние без особой динамики, нарастает одышка. По линии СМП направлен на стационарное лечение в Клинику БГМУ, где проведено КТ ОГК выявлены изменения в легких с обеих сторон по типу матового стекла (18.12.2020 - 52%). Госпитализируется для консервативного лечения в Инфекционный госпиталь Клиники БГМУ.",удовлетворительное,с улучшением


In [11]:
from typing import Optional, List, Tuple
from collections import namedtuple

In [22]:
from typing import Optional, List, Dict
from collections import defaultdict

from pymorphy2.shapes import is_roman_number
from ipymarkup import show_span_box_markup  as show_markup
from yargy import (
    Parser
)
from IPython.display import display

ROMAN = {
    'I': 1,
    'V': 5,
    'X': 10,
    'L': 50,
    'C': 100,
    'D': 500,
    'M': 1000
}


def roman_to_int(S: str) -> Optional[int]:
    """
    Перевод римских чисел в арабские.
    """
    if is_roman_number(S):
        summ = 0
        for i in range(len(S) - 1, -1, -1):
            num = ROMAN.get(S[i].upper())
            if 3 * num < summ:
                summ = summ - num
            else:
                summ = summ + num
        return summ

    return None


def show_matches(rule, *lines):
    """
    Вывод найденных сущностей 
    """
    parser = Parser(rule)
    for line in lines:
        try:
            matches = parser.findall(line)
            if matches:
                matches = sorted(matches, key=lambda _: _.span)
                spans = [_.span for _ in matches]
                show_markup(line, spans)
                facts = [_.fact for _ in matches]
                if len(facts) == 1:
                    facts = facts[0]
                display(facts)
        except Exception as e:
            print(f'Строка: {line}.\n {e} \n{list(matches)}')
            raise


def join_spans(text, spans):
    spans = sorted(spans)
    return ' '.join(
        text[start:stop]
        for start, stop in spans
    )


class Match(object):
    def __init__(self, facts, spans):
        self.facts = facts
        self.spans = spans


class Extractor:
    def __init__(self, RULE):
        self.parser = Parser(RULE)

    def __call__(self, line):
        matches = self.parser.findall(line)
        spans = [_.span for _ in matches]
        line = join_spans(line, spans)
        matches = self.parser.findall(line)
        matches = sorted(matches, key=lambda _: _.span)
        facts = [_.fact for _ in matches]
        return Match(facts, spans)


def update(main, fact, features=None):
    if features is None:
        features = []

    for key, value in fact.items():
        if key in features:
            main[key].append(value)
        else:
            main[key] = value


def show_matches_extractor(rule, features, *lines):
    extractor = Extractor(rule)
    for line in lines:
        match = extractor(line)
        show_markup(line, match.spans)
        if match.facts:
            main_fact = defaultdict(list)
            for fact in match.facts:
                fact_json = fact.as_json
                print(fact_json)
                update(main_fact, fact_json, features)
            print(main_fact)


def matches_extractor(extractor: Extractor, text: str, show = False) -> List[Dict]:
    main_fact = []
    match = extractor(text)
    if show:
        show_markup(text, match.spans)
    if match.facts:
        for fact in match.facts:
            fact_json = fact.as_json
            main_fact.append(fact_json)

    return main_fact


In [24]:
import re

from yargy.predicates import (
    eq, type as type_, gram,
    in_, dictionary
)
from yargy import (
    or_, rule
)
from yargy import interpretation as interp
from yargy.interpretation import fact



#CONSTANTS
INT = type_('INT')
NOUN = gram('NOUN')
ADJF = gram('ADJF')
PRTF = gram('PRTF')
GENT = gram('gent')
NUMR = gram('NUMR')
NUMB = gram('NUMB')
PNCT = type_('PUNCT')
LATIN = type_('LATIN')
DOT = eq('.')
COLON = eq(':')


#NUMBERS
def normalize_float(value):
    value = re.sub('[\s,.]+', '.', value)
    return float(value)

FLOAT = rule(
    INT,
    in_('.,'),
    INT
).interpretation(
    interp.custom(normalize_float)
)

DIGIT = INT.interpretation(
    interp.custom(int)
)

ROMN = rule(
    LATIN
).interpretation(
    interp.custom(roman_to_int)
)

LITERALS = {
    'один': 1,
    'два': 2,
    'три': 3,
    'четыре': 4,
    'пять': 5,
    'шесть': 6,
    'семь': 7,
    'восемь': 8,
    'девять': 9,
    'десять': 10
}

LITERAL = dictionary(LITERALS).interpretation(
    interp.normalized().custom(LITERALS.get)
)

Range = fact(
    'Range',
    ['start', 'stop']
)

VALUE = or_(
    DIGIT,
    FLOAT,
    LITERAL,
    ROMN
)

RANGE = or_(
    rule(
        VALUE.interpretation(Range.start),
        '-',
        VALUE.interpretation(Range.stop),
    ),
    rule(
         VALUE.interpretation(Range.start),
        '-',
        VALUE,
        '-',
        VALUE.interpretation(Range.stop)
    )
).interpretation(
    Range
)

Amount = fact(
    'Amount',
    ['value']
)

AMOUNT = or_(
    VALUE,
    RANGE
).interpretation(
    Amount.value
).interpretation(
    Amount
)

In [25]:
from typing import Optional

from yargy import (
    or_, rule, not_
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, in_
)
from yargy.interpretation import fact

import pymorphy2



with open('D:\\University\\вкр магистр\\Исходные\\Epicrises\\example\\data\\diseases_сustom.txt', encoding='utf-8') as f:
    diseases = f.read()
    diseases = diseases.split('\n')

Diseases = morph_pipeline(
    diseases
)
DISEASES = rule(Diseases)

MORPH = pymorphy2.MorphAnalyzer()


# Правила для основного диагноза
MainDiagnosis = fact(
    'MainDiagnosis',
    ['form', 'virus', 'diseases']
)

virus_identified = (
    'идентифицирован',
    'подтвержденная',
    'подтверждённая',
    'U07.1'
)
VIRUS_IDENTIFIED = morph_pipeline(
    virus_identified
).interpretation(MainDiagnosis.virus.const(True))

virus_not_identified = (
    'не идентифицирован',
    'неидентифицирован',
    'неиндентифицирован',
    'U07.2',    
    'неподтвержденная',
    'неподтверждённая',
    'не подтверждённая',
    'не подтвержденная',
    'подозрение'
)
VIRUS_NOT_IDENTIFIED = morph_pipeline(
    virus_not_identified
).interpretation(MainDiagnosis.virus.const(False))

# Правило для подтверждения ковида (True, False)
VIRUS = or_(
    VIRUS_IDENTIFIED,
    VIRUS_NOT_IDENTIFIED
).interpretation(MainDiagnosis.virus)


# Правило для поиска заболеваний
DISEASES_FOR_MAIN = DISEASES.interpretation(MainDiagnosis.diseases)

Forms_dict = {
    'легкий': 1,
    'лёгкий': 1,
    'средний': 2,
    'тяжелый': 3,
    'тяжёлый': 3
}
Forms_set = set(Forms_dict.keys())

Forms = morph_pipeline(
    Forms_dict.keys()
)

def forms_morph(form: str) -> Optional[int]:
    """
    Поиск пересечения нормальных форм стадий ковида с нормальной формой слова из текста
    """
    parsers = MORPH.parse(form)
    normal_forms = {parser.normal_form for parser in parsers}
    intersection = Forms_set & normal_forms
    if intersection:
        return Forms_dict[list(intersection)[0]]
    print(f'НЕ НАЙДЕНА НОРМАЛЬНАЯ ФОРМА. word: {form}, forms: {normal_forms}')
    return None

FORM_RANGE = rule(
        Forms.interpretation(Range.start.normalized().custom(forms_morph)),
        not_(eq(Forms)).optional(),
        Forms.interpretation(Range.stop.normalized().custom(forms_morph)),
).interpretation(
    Range
)

FORM = rule(Forms).interpretation(
    Range.start.normalized().custom(forms_morph)
).interpretation(
    Range
)

# Правило для форм ковида
FORMS = or_(
    FORM,
    FORM_RANGE
).interpretation(
    MainDiagnosis.form
)

MAIN_DIAGNOSIS = or_(
    DISEASES_FOR_MAIN,
    FORMS,
    VIRUS
).interpretation(
    MainDiagnosis
)


# Осложнение в диагнозе
СomplicationDiagnosis = fact(
    'СomplicationDiagnosis',
    ['diseases', 'form', 'respiratory_failure']
)

DISEASES_FOR_COMPLICATION = DISEASES.interpretation(СomplicationDiagnosis.diseases)

FORMS = or_(
    FORM,
    FORM_RANGE
).interpretation(
    СomplicationDiagnosis.form
)

DN = morph_pipeline(['дн', 'дыхательная недостаточность'])

RESPIRATORY_FAILURE = rule(
    DN,
    not_(eq(AMOUNT)).optional(), 
    AMOUNT.interpretation(СomplicationDiagnosis.respiratory_failure),
    not_(eq(AMOUNT)).optional()
)

СOMPLICATION_DIAGNOSIS = or_(
        DISEASES_FOR_COMPLICATION,    
        FORMS,
        RESPIRATORY_FAILURE
).interpretation(СomplicationDiagnosis)


# Сопутствующий диагноз
ConcomitantDiagnosis = fact(
    'ConcomitantDiagnosis',
    ['diseases', 'info']
)

DISEASES_FOR_CONCOMITANT = DISEASES.interpretation(ConcomitantDiagnosis.diseases)

CONCOMITANT_DIAGNOSIS = or_(
    rule(
        DISEASES_FOR_CONCOMITANT
    ),
    rule(
        DISEASES_FOR_CONCOMITANT,
        not_(eq(DISEASES_FOR_CONCOMITANT)).repeatable().interpretation(ConcomitantDiagnosis.info),
        DISEASES_FOR_CONCOMITANT.optional()
    ),
    rule(
        not_(eq(DISEASES_FOR_CONCOMITANT)).repeatable().interpretation(ConcomitantDiagnosis.info),
        DISEASES_FOR_CONCOMITANT,
        in_('.,').optional()
    )
   ).interpretation(ConcomitantDiagnosis)


In [26]:
import re


def make_translation(source, target):
    assert len(source) == len(target)
    return {
        ord(a): ord(b)
        for a, b in zip(source, target)
    }


DASHES_TRANSLATION = make_translation(
    '‑–—−',
    '----'
)


def delete_punct(text):
    punct = '[\\n\\xa0¬\\xad()]'
    text = re.sub(punct, ' ', text)
    text = re.sub(' +', ' ', text)
    text = re.sub('ё', 'е', text)
    text = re.sub('Ё', 'Е', text)
    return text


def preprocess(text):
    text = delete_punct(text)
    text = text.translate(DASHES_TRANSLATION)
    # text = tokenize_text(text)

    return text


In [27]:
from typing import List, Tuple, Dict
from collections import namedtuple, defaultdict
import re
from multiprocessing import Process, Manager

import pandas as pd
from pandas import DataFrame



class ExtractionDiagnosis:

    def __init__(self, diagnosis: List[str]) -> None:
        self.diagnosis = diagnosis
    
    def preprocessing(self) -> List[str]:
        for index in range(len(self.diagnosis)):
            if 'U07.2' in self.diagnosis[index]:
                find_diagnosis = self.diagnosis[index].find('U07.2')
                self.diagnosis[index] = self.diagnosis[index][:find_diagnosis + 5] + ' ' + self.diagnosis[index][find_diagnosis + 5:]
                
            if 'среднетяжелая' in self.diagnosis[index]:
                self.diagnosis[index] = self.diagnosis[index].replace('среднетяжелая', 'средне-тяжелая')
    
        clean_texts = [preprocess(text) for text in self.diagnosis]
        
        return clean_texts

    @staticmethod
    def split_texts(clean_texts: List[str]) -> Tuple[List[str], List[str], List[str]]:
        Diagnosis_text = namedtuple('Diagnosis', 'ind main complication concomitant', defaults=[None, '', '', ''])
        split_text = []
        for ind, text in enumerate(clean_texts):
            _diagnosis =  Diagnosis_text(ind, *re.split('Соп|Осл', text, maxsplit=2))
            split_text.append(_diagnosis)

        main_texts = [text.main for text in split_text]
        complication_texts = [text.complication for text in split_text]
        concomitant_texts = [text.concomitant for text in split_text]
        
        for ind in range(len(concomitant_texts)):
            concomitant_texts[ind] = re.sub('утствующий:', '', concomitant_texts[ind])
            concomitant_texts[ind] = re.sub('утствующие:', '', concomitant_texts[ind])
            
        for ind in range(len(complication_texts)):
            complication_texts[ind] = re.sub('ожнение:', '', complication_texts[ind])
            
        return main_texts, complication_texts, concomitant_texts
    
    @staticmethod
    def show_extract_main_diagnosis(texts: List[str]) -> None:
        extractor = Extractor(MAIN_DIAGNOSIS)
        facts = []
        for text in texts:
            facts_by_text = matches_extractor(extractor, text, True)
            main_fact = defaultdict(list)
            for fact in facts_by_text:
                update(main_fact, fact, ['diseases'])

            if main_fact.get('form'):
                new_form = str(main_fact['form']['start'])
                if main_fact['form'].get('stop'):
                    new_form += f"-{str(main_fact['form']['stop'])}"
                main_fact['form'] = new_form
                
            print(dict(main_fact))
            facts.append(main_fact)
            
    @staticmethod
    def show_extract_complication_diagnosis(texts: List[str]) -> None:
        extractor = Extractor(СOMPLICATION_DIAGNOSIS)
        facts = []
        for text in texts:
            facts_by_text = matches_extractor(extractor, text, True)
            main_fact = defaultdict(list)
            for fact in facts_by_text:
                update(main_fact, fact, ['diseases'])

            if main_fact.get('form'):
                new_form = str(main_fact['form']['start'])
                if main_fact['form'].get('stop'):
                    new_form += f"-{str(main_fact['form']['stop'])}"
                main_fact['form'] = new_form

            if main_fact.get('respiratory_failure'):
                value = main_fact['respiratory_failure']['value']
                if isinstance(value, int):
                    respiratory_failure = str(value)
                elif isinstance(value, dict):
                    respiratory_failure = str(value['start'])
                    respiratory_failure += f"-{str(value['stop'])}"
                else:
                    raise ValueError(f"respiratory_failure type: {type(value)}, value: {value}")

                main_fact['respiratory_failure'] = respiratory_failure
            
            print(dict(main_fact))
            facts.append(main_fact)
            
    
    @staticmethod
    def show_extract_concomitant_diagnosis(texts: List[str]) -> None:           
        extractor = Extractor(CONCOMITANT_DIAGNOSIS)
        facts = []
        for text in texts:
            facts_by_text = matches_extractor(extractor, text, True)
            main_fact = defaultdict(list)
            for fact in facts_by_text:
                update(main_fact, fact, ['diseases', 'info'])
                
            print(dict(main_fact))
            facts.append(main_fact)
            
            
        



In [28]:
df_diagnosis = list(df_main.diagnosis.copy())
extraction = ExtractionDiagnosis(df_diagnosis)
clean_texts = extraction.preprocessing()
main_texts, complication_texts, concomitant_texts = extraction.split_texts(clean_texts)

In [29]:
df_main['main_diagnosis'] = main_texts
df_main['complication_diagnosis'] = complication_texts
df_main['concomitant_diagnosis'] = concomitant_texts

In [30]:
extraction.show_extract_main_diagnosis(sample(main_texts, 5))

{'diseases': ['Коронавирусная инфекция'], 'virus': True, 'form': '2-3'}


{'diseases': ['Подозрение на коронавирусную инфекцию'], 'form': '2-3', 'virus': False}


{'diseases': ['Коронавирусная инфекция'], 'virus': False, 'form': '2-3'}


{'diseases': ['Коронавирусная инфекция'], 'virus': False, 'form': '2-3'}


{'diseases': ['Коронавирусная инфекция'], 'virus': False, 'form': '2-3'}


In [31]:
extraction.show_extract_complication_diagnosis(sample(complication_texts, 5))

{'diseases': ['Внебольничная двусторонняя полисегментарная пневмония'], 'respiratory_failure': '1'}


{'diseases': ['Внебольничная двусторонняя полисегментарная пневмония'], 'respiratory_failure': '0'}


{'diseases': ['Внебольничная двусторонняя полисегментарная пневмония'], 'respiratory_failure': '2'}


{'diseases': ['Внебольничная двусторонняя полисегментарная пневмония'], 'respiratory_failure': '1'}


{'diseases': ['Внебольничная двусторонняя полисегментарная пневмония'], 'respiratory_failure': '0-1'}


In [38]:
extraction.show_extract_concomitant_diagnosis(sample(concomitant_texts, 5))

{}


{'diseases': ['Сахарный диабет 2 типа', 'Гипертоническая болезнь'], 'info': [', инсулиннезависимый', 'II стадия, 1 степень АГ, риск ССО 4']}


{'diseases': ['ИБС'], 'info': ['. Стенокардия напряжения ФК 2']}


{'diseases': ['Гипертоническая болезнь'], 'info': ['2 ст.,1 ст.,р.3']}


{'diseases': ['Гипертоническая болезнь', 'Гипертоническая энцефалопатия'], 'info': ['3 стадия, 2 степень, риск ССО 4.', '. ХСН 0 ст.']}


In [39]:
import json
from copy import copy

from yargy import (
    or_, rule, not_
)
from yargy.pipelines import morph_pipeline, pipeline, caseless_pipeline
from yargy.predicates import (
    eq, in_caseless
)
from yargy.interpretation import fact



with open('D:\\University\\вкр магистр\\Исходные\\Epicrises\\example\\data\\complaints.json', encoding='utf-8') as f:
    complaints = json.load(f)
    
Complaint = fact(
    'Complaint',
    list(complaints.keys())
)

temperature = copy(complaints['temperature'])
complaints.pop('temperature', None)
complaints.pop('temperature_value', None)

complaint_pipelines = []
for complaint, synonyms in complaints.items():
    complaint_pipelines.append(
        rule(morph_pipeline(synonyms)).interpretation(getattr(Complaint, complaint).normalized()).interpretation(Complaint)
    )

temperature_pipeline = morph_pipeline(temperature)
TEMPERATURE = or_(
    # rule(
    #     temperature_pipeline.interpretation(Complaint.temperature)
    # ),
    rule(
        temperature_pipeline.interpretation(Complaint.temperature),
        not_(eq(AMOUNT)).optional().repeatable(), 
        AMOUNT.optional().interpretation(Complaint.temperature),
        in_caseless('СC')
    )
)

COMPLAINT = or_(*complaint_pipelines, TEMPERATURE).interpretation(Complaint)


In [40]:
from typing import List, Tuple, Dict
from collections import namedtuple, defaultdict
import re
from multiprocessing import Process, Manager, Pool

import pandas as pd
from pandas import DataFrame




class ExtractionComplaints:

    def __init__(self, texts: List[str]):
        self.texts = texts
        self.extractor = Extractor(COMPLAINT)

    def __call__(self, *args, **kwargs) -> DataFrame:
        clean_texts = [preprocess(text) for text in self.texts]
        facts = self.get_facts(clean_texts)

        df_facts = pd.DataFrame(columns=complaints.keys())
        for fact in facts:
            #df_facts = pd.concat([df_facts, fact], ignore_index=True, axis=0)
            df_facts = df_facts.append(fact, ignore_index=True)

        return df_facts

    def get_facts(self, texts: List[str]) -> None:
        facts = []
        for text in texts:
            main_fact = self.extraction_fact(text)
            facts.append(main_fact)

        return facts

    def extraction_fact(self, text: str):
        facts_by_text = matches_extractor(self.extractor, text)
        main_fact = defaultdict(list)
        for fact in facts_by_text:
            update(main_fact, fact)

        if main_fact.get('temperature') and isinstance(main_fact['temperature'], dict):
            if isinstance(main_fact['temperature']['value'], dict):
                aver = (main_fact['temperature']['value']['start'] + main_fact['temperature']['value']['stop']) / 2
                main_fact['temperature_value'] = str(aver)
            else:
                main_fact['temperature_value'] = str(main_fact['temperature']['value'])
            main_fact['temperature'] = True

        for key in main_fact.keys():
            if key not in ('temperature_value',):
                main_fact[key] = True

        return main_fact
    
    def show_extract(self):
        for text in self.texts:
            facts_by_text = matches_extractor(self.extractor, text, True)
            main_fact = defaultdict(list)
            for fact in facts_by_text:
                update(main_fact, fact)

            if main_fact.get('temperature') and isinstance(main_fact['temperature'], dict):
                if isinstance(main_fact['temperature']['value'], dict):
                    aver = (main_fact['temperature']['value']['start'] + main_fact['temperature']['value']['stop']) / 2
                    main_fact['temperature_value'] = str(aver)
                else:
                    main_fact['temperature_value'] = str(main_fact['temperature']['value'])
                main_fact['temperature'] = True

            for key in main_fact.keys():
                if key not in ('temperature_value',):
                    main_fact[key] = True
            
            print(dict(main_fact))

In [42]:
df_complaint = list(df_main.complaint.copy())
clean_texts = [preprocess(text) for text in df_complaint]
extraction = ExtractionComplaints(sample(clean_texts, 5))

extraction.show_extract()

{'weakness': True, 'body_aches': True, 'cough': True, 'sputum': True, 'dyspnea': True, 'chest_congestion': True, 'temperature': True, 'temperature_value': '38'}


{'weakness': True, 'body_aches': True, 'cough': True, 'temperature': True, 'chills': True, 'loss_appetite': True, 'temperature_value': '38.4'}


{'dyspnea': True, 'weakness': True, 'body_aches': True, 'cough': True, 'temperature': True, 'temperature_value': '39'}


{'dyspnea': True, 'weakness': True, 'body_aches': True, 'cough': True, 'loss_appetite': True}


{'temperature': True, 'weakness': True, 'headache': True, 'temperature_value': '38'}


In [43]:
with open('D:\\University\\вкр магистр\\Исходные\\Epicrises\\example\\data\\medicaments.txt', encoding='utf-8') as f:
    medicaments = f.read()
    medicaments = medicaments.split('\n')

In [44]:
from typing import Optional

from yargy import (
    or_, rule, not_
)
from yargy.pipelines import morph_pipeline
from yargy.predicates import (
    eq, in_
)
from yargy.interpretation import fact

import pymorphy2



Medicaments = caseless_pipeline(
    medicaments
)

Anamnesis = fact(
    'Anamnesis',
    ['medicaments', 'complaints', 'lung_damage_value']
)

MEDICAMENTS = rule(Medicaments).interpretation(Anamnesis.medicaments)

with open('D:\\University\\вкр магистр\\Исходные\\Epicrises\\example\\data\\complaints.json', encoding='utf-8') as f:
    complaints_json = json.load(f)
    complaints_json.pop('temperature_value', None)
    
compalints_values = []    
for values in complaints_json.values():
    compalints_values += values

complaint_pipeline = morph_pipeline(compalints_values).interpretation(Anamnesis.complaints.normalized())

COMPLAINTS = rule(complaint_pipeline).interpretation(Anamnesis)

ANAMNESIS = or_(
    MEDICAMENTS,
    COMPLAINTS
).interpretation(Anamnesis)

In [45]:
from typing import List, Tuple, Dict
from collections import namedtuple, defaultdict
import re
from multiprocessing import Process, Manager, Pool

import pandas as pd
from pandas import DataFrame





class ExtractionAnamnesis:

    def __init__(self, texts: List[str]) -> None:
        self.texts = texts
    
    def preprocessing(self) -> List[str]:
        for index in range(len(self.texts)):
            if 'Состояние' in self.texts[index]:
                find_texts = self.texts[index].find('Состояние')
                self.texts[index] = self.texts[index][:find_texts] + ' ' + self.texts[index][find_texts:]
            
        
        clean_texts = [preprocess(text) for text in self.texts]
        
        return clean_texts
    
    
    @staticmethod
    def show_extract(texts: List[str]) -> None:
        extractor = Extractor(ANAMNESIS)
        for text in texts:
            facts_by_text = matches_extractor(extractor, text, True)
            main_fact = defaultdict(list)
            for fact in facts_by_text:
                update(main_fact, fact, ['medicaments', 'complaints'])
                
            print(dict(main_fact))
            
    

In [46]:
df_anamnesis = list(df_main.anamnesis.copy())
extraction = ExtractionAnamnesis(list(df_anamnesis))
clean_texts = extraction.preprocessing()
extraction.show_extract(sample(clean_texts, 5))

{'medicaments': ['Парацетамол'], 'complaints': ['одышка']}


{'medicaments': ['абактал', 'арбидол', 'курантил'], 'complaints': ['одышка']}


{'medicaments': ['азитромицин', 'эликвис', 'дексаметазон'], 'complaints': ['одышка']}


{'medicaments': ['аспирин'], 'complaints': ['одышка']}


{'medicaments': ['Экоклав', 'Осельтамивир', 'Галавит', 'Парацетамол'], 'complaints': ['одышка']}
